#PyTorch Intro - Autoróżniczkowanie i Graf Obliczeń - Laboratorium

Do optymalizacji parametrów (wag) sieci neuronowej podczas treningu modelu wykorzystuje się **metodę stochastycznego spadku wzdłuż gradientu**.
Gradient funkcji straty względem parametrów sieci wyznaczany jest algorytmem **propagacji wstecznej** (*back propagation*).

Sieć neuronową możemy potraktować jak złożoną funkcję mapującą wejściowe dane $x \in \mathcal{X}$ (np. obraz czy sekwencję audio) na wyjście $y \in \mathcal{Y}$ parametryzowaną zestawem parametrów (wag) $\theta$.
$$
f_{\theta}( x ) = y
$$
W przypadku $n$-klasowego klasyfikatora wyjściem z sieci jest wektor $y \in \mathbb{R}^n$ nieznormalizowanych wartości, zwanych logitami, z których możemy wyznaczyć rozkład prawdopodobieństwa klas korzystając z funkcji softmax.

W jednym kroku treningu sieci neuronowych wykonujemy:
1. **Przejście w przód** - przetworzenie zestawu wejściowych danych treningowych przez sieć i wyznaczenie wartości wynikowych $y = f_{\theta}(x)$. Następnie wyznaczenie wartości funkcji straty
$\mathcal{L}$
w oparciu o wynikową wartość z sieci i prawdziwą (docelową) wartość.
2. **Przejście w tył** (propagacja wsteczna) - wyznaczenie **gradientu funkcji  straty** $\mathcal{L}$ **względem parametrów sieci** $\theta$.
3. Krok optymalizacji parametrów sieci - zmiana w kierunku przeciwnym do gradientu.

##Przygotowanie środowiska
Upewnij się, że notatnik jest uruchomiony na maszynie z GPU. Jeśli GPU nie jest dostępne zmień typ maszyny (Runtime | Change runtime type) i wybierz T4 GPU.

In [ ]:
!nvidia-smi

Biblioteka PyTorch (`torch`) jest domyślnie zainstalowana w środowisku COLAB.

In [ ]:
import torch
import numpy as np

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Sprawdzenie dostępnego urządzenia GPU.

In [ ]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")

Instalacja pakietu torchviz do wizualizacji grafów obliczeń ([link](https://github.com/szagoruyko/pytorchviz)).

In [ ]:
!pip install -q torchviz

#Automatyczne różniczkowanie (`torch.autograd`)

**Gradient** (lub gradientowe pole wektorowe) funkcji skalarnej wielu zmiennych $
f: \mathbb{R}^D → \mathbb{R}
$ oznaczamyy
$\nabla f$ (czytaj: nabla).
W układzie współrzędnych kartezjańskich gradient jest wektorem, którego składowe są pochodnymi cząstkowymi funkcji $f$:
$$\nabla f=\left[{\frac {\partial f}{\partial x_{1}}},\dots ,{\frac {\partial f}{\partial x_{n}}}\right]$$

Niech $\mathcal{L}: \mathbb{R}^D \rightarrow \mathbb{R}$ będzie pewną funkcją straty określoną dla sieci neuronowej o $D$ parametrach (wagach).
Celem treningu sieci neuronowej jest znalezienie zestawu parametrów $\mathbf{\hat{}} \in \mathbb{R}^D$ minimalizującego wartośc funkcji straty:
$$\mathbf{\hat{w}} = \arg \min_{\textbf{w}} \mathcal{L} \left( \textbf{w} \right)$$
W metodzie **spadku wzdłuż gradientu** zaczynamy od losowo zainicjalizowanych parametrów (wag) sieci $\textbf{w}_0$ a następnie iteracyjnie aktualizujemy parametry sieci w kierunku przeciwnym do wartości gradientu:
$$
\mathbf{w}_{t+1} = \mathbf{w}_{t} - \eta \nabla \mathcal{L} \left( \mathbf{w}_t \right)
$$.





Aby wyznaczyć **gradient funkcji straty względem parametrów sieci**, PyTorch posiada wbudowany mechanizm różniczkowania o nazwie `torch.autograd`. Umożliwia on automatyczne obliczanie gradientu dla dowolnego grafu obliczeniowego.

Obiekty typu Tensor posiadają logiczną flagę `requires_grad`.
Domyślnie flaga `requires_grad` jest ustawiana na `False`.
Po jej włączeniu PyTorch będzie automatycznie budował grafy dla wszystkich obliczeń wykonanych z wykorzystaniem tego tensora aby umożliwić automatyczne wyznaczanie gradientu.
Jeśli jeden z argumentów operacji na tensorach ma ustawioną flagę `requires_grad`, wynik również będzie miał ustawioną tę flagę.

#Zadania do wykonania

##Zadanie 1

Niech $f: \mathbb{R}^2 \rightarrow \mathbb{R}$ będzie funkcją:

$$f(x) = sin(x_1) cos(x_2) + sin(0.5 \cdot x_1) cos(0.5 \cdot x_2)$$.

1.   Napisz kod wyznaczających lokalne minimum funkcji $f$ metodą spadku wzdłuż gradientu dla początkowych wartości argumentów $x_1, x_2$ wylosowanych z zakresu $[0; 10]$. Wyświetl znalezione minimum oraz wartości argumentów funcji.
    *   Wykorzystaj mechanizm autoróżniczkowania do wyznaczenia gradientu funkcji $f$. Pamiętaj, aby włączyć budowanie grafu obliczeń dla tensorów `x1` i `x2`.
    *   Liczbę iteracji i stopę uczenia dobierz eksperymentalnie.
    *   Na końcu każdego kroku optymalizacji wyzeruj wartości gradientów każdego z argumentów (`x.grad.zeros_()`). Domyślnie PyTorch akumuluje wartości gradientu dla wielu wywołań przejścia w tył `backward()`.
2.   Zwizualizuj trajektorie parametrów $(x_1, x_2)$ w kolejnych krokach optymalizacji powtarzając cały proces kilkakrotnie, rozpoczynając od losowo wybranych wartości argumentów, każdy z zakresu $[0; 10]$. Czy za każdym razem osiągane jest to samo lokalne minimum?
3.   (opcjonalnie) Zaimplementuj zwektoryzowaną wersję procedury wykonującej minimalizację wartości funkcji $f$ dla wielu zestawów argumentów wejściowych danych jako macierz (tensor) o wymiarach $(n,2)$.
   *   Zwektoryzowana wersja nie zawiera pętli przechodzącej po każdym z $n$ zestawów argumentów. W jednym kroku optymalizacji aktualizuje wszystkie $n$ zestawów argumentów.
   *   Aby wyznaczyć gradient dla każdego elementu z osobna tensora który nie jest skalarem, np. dla $n$-elementowego wektora `f` zawierającego wyniki obliczeń dla $n$ zestawów argumentów, jako argument metody `backward` podaj tensor jedynek o rozmiarze równym rozmiarowi `f`, np. `f.backward(torch.ones_like(f))`.

Wizualizacja funkcji $f(x)$ z wykorzystaniem biblioteki Plotly.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Utwórz siatkę wartości x i y
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

fig = go.Figure(data=go.Contour(z=Z, x=x, y=y, colorscale='Viridis'))
fig.update_layout(title="Izolinie funkcji 3D", xaxis_title="X", yaxis_title="Y")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y, colorscale='Viridis')])
fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
    )
fig.show()